In [1]:
import collections
import numpy as np
import pandas as pd
import re
import glob
import json

from argparse import Namespace

from torchtext.utils import download_from_url, extract_archive

In [2]:
dataset_tar = download_from_url('https://drive.google.com/uc?id=1k7GfVRqrHFK00ABkit0oGQo62fCakMSZ', root='data/')
extracted_files = extract_archive(dataset_tar)

In [3]:
args = Namespace(
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="data/faq_with_splits.csv",
    seed=1234,
    min_questions_per_category=3000
)

In [4]:
def find_files(path):
    return glob.glob(path)

def read_json(filename):
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

In [5]:
json_data = []

for file in find_files('.data/gyakori_*'):
    print("Reading file:", file)
    json_data.append(read_json(file))
    
faq = pd.DataFrame()

for data in json_data:
    frames  = [faq, pd.DataFrame(data)]
    faq = pd.concat(frames).reset_index(drop=True)

Reading file: .data/gyakori_szamitastechnika
Reading file: .data/gyakori_egeszseg
Reading file: .data/gyakori_egeszseg_20000
Reading file: .data/gyakori_allatok_14000
Reading file: .data/gyakori_szorakozas_30000


In [6]:
faq.rename(columns = {
        "valasz": "answer",
        "kategoriak": "categories",
        "hosszu_kerdes": "long_question",
        "rovid_kerdes": "short_question"
    }, inplace = True)

faq['main_category'] = faq.apply(lambda row: row.categories[0], axis = 1)
faq['sub_category'] = faq.apply(lambda row: row.categories[1], axis = 1)

faq.drop('categories', axis = 1, inplace = True)

faq.head()

,answer,long_question,short_question,keywords,main_category,sub_category
0,Várak régen is voltak. Ha mindhárom tornyot le...,Miért lett ilyen sz@r a honfoglaló? Régen tök ...,Miért lett ilyen sz@r a honfoglaló?,"[Honfoglaló, vár]",Számítástechnika,Internet
1,"Ahogy írták, az stdio az a C-s függvénykönyvtá...",C++-ban melyiket érdemesebb használni? Stdio v...,C++-ban melyiket érdemesebb használni? Stdio v...,"[C++, iostream, konzol, Windows, Visual Studio]",Számítástechnika,Programozás
2,"c++-ban olyat használsz, amilyet akarsz.",Lehet-e más programozási nyelvben megírt alapp...,Lehet-e más programozási nyelvben megírt alapp...,"[Java, programozás, C++]",Számítástechnika,Programozás
3,"Minden megfelel, amin fut az általad használni...",A 2016os Macbook Air megfelel egy programozást...,A 2016os Macbook Air megfelel egy programozást...,"[MacBook Air, egyetem, programozás]",Számítástechnika,Hardverek
4,"Felejtsd el,ennyiért nem kapni normálisat,vagy...",Mi a legjobb számítógépház? 10000ft körül lehe...,Mi a legjobb számítógépház?,"[AMD, Intel, PC, Radeon, NVIDIA]",Számítástechnika,Hardverek


In [7]:
categories = set(faq.main_category)
print(categories)

{'Egészség', 'Szórakozás', 'Számítástechnika', 'Gyerekvállalás, nevelés', 'Állatok'}


In [8]:
for category in categories:
    category_size = faq[faq.main_category == category].shape[0]
    if category_size < args.min_questions_per_category:
        faq = faq[faq.main_category != category]

In [9]:
import unidecode
def unicode_to_ascii(data):
    return unidecode.unidecode(re.sub(r"[,.;@#?!&$]+\ *", " ", data).lower())

In [10]:
faq.long_question = faq.long_question.apply(lambda x: unicode_to_ascii(x))
faq.short_question = faq.short_question.apply(lambda x: unicode_to_ascii(x))
faq.answer = faq.answer.apply(lambda x: unicode_to_ascii(x))

/home/migeran/.local/lib/python3.8/site-packages/unidecode/__init__.py:131: RuntimeWarning: Surrogate character '\ud83d' will be ignored. You might be using a narrow Python build.
  repl = _get_repl_str(char)
/home/migeran/.local/lib/python3.8/site-packages/unidecode/__init__.py:131: RuntimeWarning: Surrogate character '\ude0f' will be ignored. You might be using a narrow Python build.
  repl = _get_repl_str(char)


In [11]:
by_main_category = collections.defaultdict(list)

for _, row in faq.iterrows():
    by_main_category[row.main_category].append(row.to_dict())

In [12]:
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_main_category.items()):
    np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_proportion * n)
    n_val = int(args.val_proportion * n)
    n_test = int(args.test_proportion * n)
    
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  
    
    final_list.extend(item_list)

In [13]:
final_faq = pd.DataFrame(final_list)

In [14]:
final_faq.split.value_counts()

train    117686
test      25223
val       25217
Name: split, dtype: int64

In [15]:
final_faq.head()

,answer,long_question,short_question,keywords,main_category,sub_category,split
0,oreg 14 eves korom ota dolgozok hogy amit a cs...,irnatok nekem valami biztatot miutan par napja...,irnatok nekem valami biztatot,"[depresszió, gödör, szerelem, bánat]",Egészség,Mentális egészség,train
1,2-esre ne hallgass segiteni fog az ad maskepp ...,vehetek be az antidepresszanssal egy idoben ny...,vehetek be az antidepresszanssal egy idoben ny...,"[antidepresszáns, tea, nyugtató, gyógynövény]",Egészség,Mentális egészség,train
2,csak a hulyeseggel nem kell fekudni azt labon ...,azt halottam tudogyulladassal nem nagyon kell ...,azt halottam tudogyulladassal nem nagyon kell ...,"[tüdőgyulladás, vesegyulladás]",Egészség,Betegségek,train
3,ignorald a kozossegek ilyenek kineznek valakit...,mit csinaljak ha szekal az osztalyom a miatt h...,mit csinaljak ha szekal az osztalyom a miatt h...,"[szekálás, betegség, pajzsmirigy, hiányzás]",Egészség,"Immunrendszer, fertőzések",train
4,kedves kerdezo eloszor is nem tudom hany eves ...,hirtelen nagyon eros lett a verzesem egyik per...,hirtelen nagyon eros lett a verzesem egyik per...,"[vérzés, darabos, nőgyógyász]",Egészség,Nők egészsége,train


In [16]:
final_faq.to_csv(args.output_munged_csv, index=False)